In [1]:
import pandas as pd
import numpy as np

In [3]:
df_vehicles = pd.read_csv("data/Traffic_Crashes_Vehicles.csv")
df_people = pd.read_csv("data/Traffic_Crashes_People.csv")

C:\Users\Owner\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (20,39,40,41,43,47,48,49,52,54,57,58,60,70) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\Owner\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (19,23,24,25,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
df_final = df_people.sample(n=300000, random_state=42)
df_final = df_final.merge(df_vehicles, how="left", left_on="CRASH_RECORD_ID", right_on="CRASH_RECORD_ID")

In [12]:
df_final

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,VEHICLE_ID_x,CRASH_DATE_x,SEAT_NO,CITY,STATE,ZIPCODE,SEX,...,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS
0,O1570236,DRIVER,9f3300f82fb71bd87e1a388f1190525c5cd80462c39dc6...,1493761.0,05/06/2023 08:50:00 PM,NaN,NaN,NaN,NaN,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,O1570236,DRIVER,9f3300f82fb71bd87e1a388f1190525c5cd80462c39dc6...,1493761.0,05/06/2023 08:50:00 PM,NaN,NaN,NaN,NaN,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,O1596587,DRIVER,cc52c13e5510187a8bc57c42ef1b62b32333e908163e07...,1519222.0,06/12/2023 10:30:00 PM,NaN,CHICAGO,IL,60659,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,O1596587,DRIVER,cc52c13e5510187a8bc57c42ef1b62b32333e908163e07...,1519222.0,06/12/2023 10:30:00 PM,NaN,CHICAGO,IL,60659,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,O130199,DRIVER,012f0e85941af3ed53e800bd3fb2dee18f91b068812fb5...,130114.0,02/26/2017 12:00:00 PM,NaN,NaN,NaN,NaN,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629569,O1342669,DRIVER,c45fe908b1b5b115d909af70b71b8fecaf929c1f83ea92...,1275601.0,05/21/2022 12:50:00 PM,NaN,NaN,NaN,NaN,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
629570,O527730,DRIVER,06d845100c4a605e5407d31b63d3840ac23f1e3b89f7ec...,503988.0,11/23/2018 09:45:00 AM,NaN,CHICAGO,IL,60644,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
629571,O527730,DRIVER,06d845100c4a605e5407d31b63d3840ac23f1e3b89f7ec...,503988.0,11/23/2018 09:45:00 AM,NaN,CHICAGO,IL,60644,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
629572,P37948,PASSENGER,dd3ea43ebaf3183b021c3a4669996727a6ab0acb9c55af...,175734.0,06/30/2017 03:00:00 PM,3.0,CHICAGO,IL,60647,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_final.to_csv("data/Final_Data.csv")